# Task 4 - Fire Evacuation Planning for the MI Building

In [ ]:
import init_notebook
import warnings
from torchvision.datasets import MNIST
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import TensorDataset, DataLoader
from vae import VAETrainer, VAEConfig

warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
def plot_mi_building_points(points, title):
    x_min_max = (points.T[0].min(), points.T[0].max())
    y_min_max = (points.T[1].min(), points.T[1].max())
    img = plt.imread('mi_buiding.png')
    plt.imshow(img,zorder=0, extent=[x_min_max[0], x_min_max[1], y_min_max[0], y_min_max[1]], aspect='auto')
    plt.title(title)
    plt.xlabel("x")
    plt.ylabel("y")
    sns.scatterplot(x=points.T[0], y=points.T[1])
    plt.show()

In [ ]:
train_set = np.load("FireEvac_train_set.npy")
test_set = np.load("FireEvac_test_set.npy")

print(f"training set shape:",list(train_set.shape))
print(f"testing set shape:",list(test_set.shape))

print("training [min, max]:",f"[{train_set.min().item()}, {train_set.max().item()}]")
print("testing [min, max]:",f"[{test_set.min().item()}, {test_set.max().item()}]")

In [ ]:
plot_mi_building_points(train_set, "Training Set")

In [ ]:
plot_mi_building_points(test_set, "Testing Set")

In [ ]:
print("training [min, max]:",f"[{train_set.min().item()}, {train_set.max().item()}]")
print("testing [min, max]:",f"[{test_set.min().item()}, {test_set.max().item()}]")

In [ ]:
train_set_tensor = torch.from_numpy(train_set)
train_dataset = TensorDataset(train_set_tensor, train_set_tensor)

test_set_tensor = torch.from_numpy(test_set)

test_dataset = TensorDataset(test_set_tensor, test_set_tensor)

## Model Tarining

In [ ]:
config = VAEConfig(
    input_dim=2,
    latent_dim=2,
    encoder_layers=[64, 64],
    decoder_layers=[64, 64],
    learning_rate=0.005,
    batch_size=64,
    epochs=400,
    visualization_interval=None,
    output_linear = True
)

vae_trainer = VAETrainer(config=config, train_set=train_dataset, test_set=test_dataset)
vae_trainer.train()

## Reconstructed Test Set

In [ ]:
test_predicted_scaled = vae_trainer.predict_test()
# test_predicted = scaler.inverse_transform(test_predicted_scaled)
# test_predicted = (test_predicted_scaled + 1) * ((train_set.max() - train_set.min()) / 2) + train_set.min()

plot_mi_building_points(test_predicted_scaled, "Reconstructed Test Set")

## Generated Samples

In [ ]:
generated_samples = vae_trainer.get_n_generated_samples(1000)
generated_samples = generated_samples * 180/ generated_samples.max()

plot_mi_building_points(generated_samples, "Generated samples")

## Can the critical point be reached?

In [ ]:
def points_inside_box(tensor):
    critical_area_ll = np.array([130.0, 50.0])
    critical_area_ur = np.array([150.0, 70.0])

    inidx = np.all((critical_area_ll <= tensor.numpy()) & (tensor.numpy() <= critical_area_ur) , axis=1)
    return tensor[inidx]

In [ ]:
for i in range(1, 100000, 50):
    generated_samples = vae_trainer.get_n_generated_samples(i)
    points_in = points_inside_box(generated_samples)
    if len(points_in) >= 100:
        print(i)
        break
else:
    print("Cannot be reached")
